In [1]:
from train import Trainer, TrainArgs, load_model
from model import ModelArgs, ModelArgsIterator, create_model_from_data_generator
from dataset import BalancedParenthesisDataGenerator

In [2]:
train_args_test = TrainArgs(epochs=1, trainset_size=4*1024 ,valset_size=1024)
train_args_full = TrainArgs(epochs=10)

## Parenthesis Balancer

In [ ]:
data_gen = BalancedParenthesisDataGenerator(n_ctx_numeric=20)
model_args = ModelArgs(n_layers=2, n_heads=2, d_model=48)

model = create_model_from_data_generator(data_gen, model_args)
trainer = Trainer(model, data_gen, train_args_full)
trainer.train()

In [4]:
data_gen = BalancedParenthesisDataGenerator(n_ctx_numeric=20)
model_args_iterator = ModelArgsIterator(
    n_layers=[2, 3],
    n_heads=[1, 2],
    d_model=[32, 64, 256],
    attn_only=[False]
)

for model_args in model_args_iterator:
    model = create_model_from_data_generator(data_gen, model_args)
    trainer = Trainer(model, data_gen, train_args_full)
    trainer.train()
    trainer.save_model(model_args, task_name='bal_paren_20')

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

...

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
data_gen = BalancedParenthesisDataGenerator(n_ctx_numeric=20)
model_args = ModelArgs(n_layers=1, n_heads=2, d_model=64)
model = create_model_from_data_generator(data_gen, model_args)

trainer = Trainer(model, data_gen, train_args_full)
trainer.train()

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

...

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

  0%|          | 0/195 [00:00<?, ?it/s]

AttributeError: 'Trainer' object has no attribute 'save_model'